In [2]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

'.env_file' loaded!
ENV 'PROD' configured!


In [3]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_ecd_new_empresas"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

2025-11-15T23:47:44.442485Z [info     ] Using json file settings.      [root] loc=spark_utils_session.py:301
2025-11-15T23:47:44.443611Z [info     ] Exporting default ENV.         [root] loc=spark_utils_session.py:305
2025-11-15T23:47:44.444153Z [info     ] Exporting custom ENVs.         [root] loc=spark_utils_session.py:338
2025-11-15T23:47:44.444498Z [info     ] Building profile 'efd_t2'.     [root] loc=spark_utils_session.py:221
2025-11-15T23:47:44.444869Z [info     ] Not enough info for building the kerberos client. Ignoring it [root] loc=spark_utils_session.py:284


:: loading settings :: url = jar:file:/opt/cloudera/parcels/SPARK3-3.5.4.3.5.7191000.0-30-1.p0.68499982/lib/spark3/jars/ivy-2.5.2.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tsevero/.ivy2/cache
The jars for the packages stored in: /home/tsevero/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.oracle.database.security#oraclepki added as a dependency
com.oracle.database.security#osdt_core added as a dependency
com.oracle.database.security#osdt_cert added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dbc19d31-37de-4541-bae2-95906e93eb37;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found com.oracle.database.security#oraclepki;21.19.0.0 in central
	found com.oracle.databa

In [4]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

Hive Session ID = cda11462-c57d-4a52-a7a8-d9c77dd6e76e


+------------------+
|namespace         |
+------------------+
|anac              |
|bcadastro         |
|bpe               |
|c115              |
|ccc               |
|ccg               |
|cte               |
|default           |
|destda            |
|detran_share      |
|dime              |
|due               |
|efd               |
|fci               |
|gecob             |
|gescol            |
|gessimples        |
|gplam             |
|information_schema|
|malhas            |
+------------------+
only showing top 20 rows



In [5]:
# ============================================================================
# CÉLULA 1: CONFIGURAÇÃO INICIAL - PROJETO ECD
# ============================================================================
# Escrituração Contábil Digital - Análise Completa
# Auditor Fiscal da Receita Estadual de SC
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    coalesce as spark_coalesce,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    lit as spark_lit,
    concat as spark_concat,
    expr as spark_expr,
    abs as spark_abs
)

# Machine Learning imports
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score
)
import xgboost as xgb

# Acesso ao SparkSession
spark = session.sparkSession

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Configuração de estilo para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Verificar se spark existe na sessão
try:
    spark
    print("✅ Sessão Spark já ativa!")
except NameError:
    print("⚠️ Sessão Spark não encontrada. Por favor, inicialize a sessão Spark.")

print("\n" + "="*80)
print("🚀 PROJETO ECD - ANÁLISE EXPLORATÓRIA E MACHINE LEARNING")
print("="*80)
print(f"📅 Data de execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("📊 Bibliotecas carregadas com sucesso!")
print("="*80 + "\n")

✅ Sessão Spark já ativa!

🚀 PROJETO ECD - ANÁLISE EXPLORATÓRIA E MACHINE LEARNING
📅 Data de execução: 15/11/2025 20:48:03
📊 Bibliotecas carregadas com sucesso!



In [30]:
# ================================================================================
# APLICAR MODELO ML EM TODAS AS CONTAS DO ECD - VERSÃO CORRIGIDA
# ================================================================================

print("🚀 APLICANDO MODELO ML EM TODAS AS CONTAS DO ECD")
print("=" * 80)

from pyspark.sql.functions import (
    col, when, lit, log1p, abs as spark_abs, length, lower,
    regexp_extract, substring, coalesce
)
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassificationModel
import time

# ================================================================================
# CONFIGURAÇÕES
# ================================================================================

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'
UF_FILTRO = 'SC'
ANO_REFERENCIA = 2024

print(f"\n⚙️  Configurações:")
print(f"   Database ML: {DATABASE}")
print(f"   Database ECD: {DATABASE_ECD}")
print(f"   UF: {UF_FILTRO}")
print(f"   Ano: {ANO_REFERENCIA}")

# ================================================================================
# 1. CARREGAR MODELO TREINADO
# ================================================================================

print(f"\n📥 Carregando modelo treinado...")

try:
    model_path = f"/user/{spark.sparkContext.sparkUser()}/models/rf_final_producao_{UF_FILTRO}_{ANO_REFERENCIA}"
    modelo = RandomForestClassificationModel.load(model_path)
    print(f"✅ Modelo FINAL carregado: {model_path}")
except:
    try:
        model_path = f"/user/{spark.sparkContext.sparkUser()}/models/rf_abordagem_a_{UF_FILTRO}_{ANO_REFERENCIA}"
        modelo = RandomForestClassificationModel.load(model_path)
        print(f"✅ Modelo Abordagem A carregado: {model_path}")
    except Exception as e:
        print(f"❌ Erro ao carregar modelo: {e}")
        raise

# ================================================================================
# 2. CARREGAR TODAS AS CONTAS DO ECD - SEM TABELA DE IDENTIFICAÇÃO
# ================================================================================

print(f"\n📥 Carregando TODAS as contas do ECD (Balanço Patrimonial)...")

df_todas_contas = spark.sql(f"""
    SELECT 
        bp.id_ecd,
        CAST(bp.id_ecd AS STRING) as cnpj,  -- Usar id_ecd como proxy temporário
        bp.cod_agl as cd_conta,
        bp.descr_cod_agl as descr_conta,
        bp.cod_agl_sup as cd_conta_sup,
        bp.nivel_agl as nivel_conta,
        bp.ind_grp_bal,
        'BP' as origem_demonstrativo,
        CAST(NULL AS STRING) as ind_grp_dre,  -- BP não tem ind_grp_dre
        bp.vl_cta_fin as vl_conta_final,
        bp.vl_cta_ini as vl_conta_inicial,
        bp.ind_dc_cta_fin,
        bp.ind_dc_cta_ini,
        pc.cd_natureza,
        pc.tp_conta as tp_conta_pc,
        CASE 
            WHEN bp.nivel_agl = 1 THEN 'S'
            WHEN bp.nivel_agl > 1 THEN 'A'
            ELSE 'A'
        END as tp_conta_agl
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
    LEFT JOIN {DATABASE_ECD}.ecd_ri050_plano_contas pc
        ON bp.id_ecd = pc.id_ecd
        AND bp.cod_agl = pc.cd_conta_anl
""")

total_contas = df_todas_contas.count()
print(f"✅ Total de contas carregadas: {total_contas:,}")

# ================================================================================
# 3. CRIAR FEATURES
# ================================================================================

print(f"\n🔧 Criando features...")
inicio_features = time.time()

# Features básicas de texto
df_features = df_todas_contas.withColumn(
    'tamanho_descricao',
    length(col('descr_conta'))
).withColumn(
    'tamanho_codigo',
    length(col('cd_conta'))
).withColumn(
    'tem_ponto_codigo',
    when(col('cd_conta').contains('.'), 1.0).otherwise(0.0)
).withColumn(
    'tem_hifen_codigo',
    when(col('cd_conta').contains('-'), 1.0).otherwise(0.0)
).withColumn(
    'tem_underscore_codigo',
    when(col('cd_conta').contains('_'), 1.0).otherwise(0.0)
).withColumn(
    'primeiro_digito_codigo',
    substring(col('cd_conta'), 1, 1)
)

# Features de valores
df_features = df_features.withColumn(
    'valor_absoluto_final',
    spark_abs(coalesce(col('vl_conta_final'), lit(0)))
).withColumn(
    'valor_absoluto_inicial',
    spark_abs(coalesce(col('vl_conta_inicial'), lit(0)))
).withColumn(
    'variacao_valor',
    coalesce(col('vl_conta_final'), lit(0)) - coalesce(col('vl_conta_inicial'), lit(0))
)

# Features de hierarquia
df_features = df_features.withColumn(
    'tem_conta_superior',
    when(col('cd_conta_sup').isNotNull(), 1.0).otherwise(0.0)
)

# Features de palavras-chave
descr_lower = lower(col('descr_conta'))

df_features = df_features.withColumn(
    'tem_palavra_caixa',
    when(descr_lower.contains('caixa'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_banco',
    when(descr_lower.contains('banco'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_estoque',
    when(descr_lower.contains('estoque'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_cliente',
    when(descr_lower.contains('cliente'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_fornecedor',
    when(descr_lower.contains('fornecedor'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_salario',
    when(descr_lower.contains('salario') | descr_lower.contains('salário'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_tributo',
    when(descr_lower.contains('tributo') | descr_lower.contains('imposto'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_receita',
    when(descr_lower.contains('receita'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_despesa',
    when(descr_lower.contains('despesa'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_financeiro',
    when(descr_lower.contains('financeiro') | descr_lower.contains('financeira'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_imobilizado',
    when(descr_lower.contains('imobilizado'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_capital',
    when(descr_lower.contains('capital'), 1.0).otherwise(0.0)
).withColumn(
    'tem_palavra_lucro',
    when(descr_lower.contains('lucro') | descr_lower.contains('prejuízo') | descr_lower.contains('prejuizo'), 1.0).otherwise(0.0)
)

# Features de interação
df_features = df_features.withColumn(
    'razao_valor_final_inicial',
    when(col('valor_absoluto_inicial') > 0,
         col('valor_absoluto_final') / col('valor_absoluto_inicial')
    ).otherwise(lit(0.0))
).withColumn(
    'nivel_x_tamanho_descricao',
    col('nivel_conta') * col('tamanho_descricao')
).withColumn(
    'nivel_x_tamanho_codigo',
    col('nivel_conta') * col('tamanho_codigo')
).withColumn(
    'log_valor_absoluto_final',
    log1p(col('valor_absoluto_final'))
).withColumn(
    'log_valor_absoluto_inicial',
    log1p(col('valor_absoluto_inicial'))
).withColumn(
    'diferenca_absoluta_valores',
    spark_abs(col('valor_absoluto_final') - col('valor_absoluto_inicial'))
)

tempo_features = time.time() - inicio_features
print(f"✅ Features criadas em {tempo_features:.2f}s")

# ================================================================================
# 4. CRIAR CLASSIFICAÇÃO NIVEL 1
# ================================================================================

print(f"\n🔧 Criando classificação Nível 1...")

df_features = df_features.withColumn(
    'classificacao_nivel1',
    when(col('ind_grp_bal') == 'A', 'ATIVO')
    .when(col('ind_grp_bal') == 'P', 'PASSIVO')
    .otherwise('OUTROS')
)

# Preencher ind_grp_dre como 'N/A' se nulo
df_features = df_features.withColumn(
    'ind_grp_dre',
    coalesce(col('ind_grp_dre'), lit('N/A'))
)

# ================================================================================
# 5. STRING INDEXERS
# ================================================================================

print(f"\n🔧 Indexando variáveis categóricas...")

categorical_cols = [
    'origem_demonstrativo', 'ind_grp_bal', 'ind_grp_dre',
    'cd_natureza', 'tp_conta_agl', 'tp_conta_pc',
    'primeiro_digito_codigo', 'classificacao_nivel1'
]

df_indexed = df_features

for col_name in categorical_cols:
    indexer = StringIndexer(
        inputCol=col_name,
        outputCol=f'{col_name}_index',
        handleInvalid='keep'
    )
    df_indexed = indexer.fit(df_indexed).transform(df_indexed)

print(f"✅ Variáveis indexadas")

# ================================================================================
# 6. VETOR DE FEATURES
# ================================================================================

print(f"\n🔧 Montando vetor de features...")

numeric_features = [
    'nivel_conta', 'tamanho_descricao', 'tamanho_codigo',
    'tem_ponto_codigo', 'tem_hifen_codigo', 'tem_underscore_codigo',
    'valor_absoluto_final', 'valor_absoluto_inicial', 'variacao_valor',
    'tem_conta_superior',
    'tem_palavra_caixa', 'tem_palavra_banco', 'tem_palavra_estoque',
    'tem_palavra_cliente', 'tem_palavra_fornecedor', 'tem_palavra_salario',
    'tem_palavra_tributo', 'tem_palavra_receita', 'tem_palavra_despesa',
    'tem_palavra_financeiro', 'tem_palavra_imobilizado', 'tem_palavra_capital',
    'tem_palavra_lucro',
    'razao_valor_final_inicial', 'nivel_x_tamanho_descricao', 'nivel_x_tamanho_codigo',
    'log_valor_absoluto_final', 'log_valor_absoluto_inicial', 'diferenca_absoluta_valores'
]

categorical_features = [f'{col}_index' for col in categorical_cols]
all_features = numeric_features + categorical_features

assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features_v2',
    handleInvalid='keep'
)

df_assembled = assembler.transform(df_indexed)
print(f"✅ Vetor de features montado")

# ================================================================================
# 7. FAZER PREDIÇÕES EM LOTES (PARA EVITAR TIMEOUT)
# ================================================================================

print(f"\n🔮 Aplicando modelo em TODAS as contas...")
print(f"   Processando em lotes para evitar timeout...")

# Criar partições menores
df_assembled = df_assembled.repartition(200)  # Ajuste conforme necessário

inicio_pred = time.time()
df_predicoes = modelo.transform(df_assembled)

# Persistir para acelerar operações futuras
df_predicoes = df_predicoes.persist()

tempo_pred = time.time() - inicio_pred
print(f"✅ Predições concluídas em {tempo_pred:.2f}s ({tempo_pred/60:.2f} min)")

# ================================================================================
# 8. MAPEAR LABELS
# ================================================================================

print(f"\n🔧 Mapeando labels para classificações...")

# Carregar mapeamento
mapeamento = spark.sql(f"""
    SELECT DISTINCT label, classificacao_nivel2
    FROM {DATABASE}.ecd_ml_dataset
    ORDER BY label
""").collect()

label_to_class = {row['label']: row['classificacao_nivel2'] for row in mapeamento}

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(returnType=StringType())
def map_label_to_class(label):
    if label is None:
        return None
    return label_to_class.get(int(label), 'NAO_CLASSIFICADO')

df_predicoes_final = df_predicoes.withColumn(
    'classificacao_nivel2_pred',
    map_label_to_class(col('prediction'))
)

print(f"✅ Labels mapeadas")

# ================================================================================
# 9. SALVAR
# ================================================================================

print(f"\n💾 Salvando predições COMPLETAS...")

df_save = df_predicoes_final.select(
    'id_ecd',
    'cnpj',
    'cd_conta',
    'descr_conta',
    'classificacao_nivel2_pred',
    'prediction',
    'probability'
).withColumnRenamed('classificacao_nivel2_pred', 'classificacao_nivel2')

df_save.createOrReplaceTempView("temp_predictions_all")

spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_predictions_ALL")

spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_predictions_ALL
    STORED AS PARQUET
    AS SELECT * FROM temp_predictions_all
""")

total_pred_salvas = df_save.count()
print(f"✅ Salvo: {DATABASE}.ecd_ml_predictions_ALL")
print(f"   Total: {total_pred_salvas:,}")

# ================================================================================
# 10. ESTATÍSTICAS
# ================================================================================

print(f"\n📊 ESTATÍSTICAS")
print("=" * 80)

print(f"\n📊 Top 20 classificações:")
df_save.groupBy('classificacao_nivel2').count() \
    .orderBy(col('count').desc()) \
    .limit(20) \
    .show(truncate=False)

# Comparação
pred_antes = 246359
pred_depois = total_pred_salvas

print(f"\n📊 COMPARAÇÃO:")
print(f"   Antes (teste): {pred_antes:,}")
print(f"   Depois (ALL):  {pred_depois:,}")
print(f"   GANHO:         +{pred_depois - pred_antes:,}")

# Unpersist
df_predicoes.unpersist()

print("\n" + "=" * 80)
print("✅ CONCLUÍDO!")
print("=" * 80)
print(f"\n🎯 Agora execute os Scripts 1, 2 e 3 usando:")
print(f"   {DATABASE}.ecd_ml_predictions_ALL")
print("=" * 80)

🚀 APLICANDO MODELO ML EM TODAS AS CONTAS DO ECD

⚙️  Configurações:
   Database ML: neac
   Database ECD: usr_sat_ecd
   UF: SC
   Ano: 2024

📥 Carregando modelo treinado...


✅ Modelo FINAL carregado: /user/tsevero/models/rf_final_producao_SC_2024

📥 Carregando TODAS as contas do ECD (Balanço Patrimonial)...


✅ Total de contas carregadas: 144,602,905

🔧 Criando features...
✅ Features criadas em 0.16s

🔧 Criando classificação Nível 1...

🔧 Indexando variáveis categóricas...


✅ Variáveis indexadas

🔧 Montando vetor de features...
✅ Vetor de features montado

🔮 Aplicando modelo em TODAS as contas...
   Processando em lotes para evitar timeout...
✅ Predições concluídas em 0.20s (0.00 min)

🔧 Mapeando labels para classificações...
✅ Labels mapeadas

💾 Salvando predições COMPLETAS...


✅ Salvo: neac.ecd_ml_predictions_ALL
   Total: 144,602,905

📊 ESTATÍSTICAS

📊 Top 20 classificações:


[Stage 2370:>                                                       (0 + 1) / 1]

+------------------------+--------+
|classificacao_nivel2    |count   |
+------------------------+--------+
|ATIVO_CIRCULANTE        |74873652|
|PASSIVO_CIRCULANTE      |60477546|
|PASSIVO                 |3749043 |
|ATIVO                   |1886263 |
|DISPONIBILIDADES        |1001849 |
|CAPITAL_SOCIAL          |907726  |
|FORNECEDORES            |679718  |
|ESTOQUES                |286137  |
|TRIBUTOS_A_PAGAR        |216264  |
|IMOBILIZADO             |161367  |
|CREDITOS                |155556  |
|INVESTIMENTOS           |140849  |
|RECEITA_BRUTA           |35910   |
|RESERVAS                |20245   |
|CUSTOS                  |7126    |
|PATRIMONIO_LIQUIDO      |3608    |
|OUTRAS_RECEITAS_DESPESAS|46      |
+------------------------+--------+


📊 COMPARAÇÃO:
   Antes (teste): 246,359
   Depois (ALL):  144,602,905
   GANHO:         +144,356,546

✅ CONCLUÍDO!

🎯 Agora execute os Scripts 1, 2 e 3 usando:
   neac.ecd_ml_predictions_ALL


In [33]:
# ================================================================================
# SCRIPT 1: ANÁLISE COMPLETA - VERSÃO FINAL CORRIGIDA
# ================================================================================

print("🔍 ANÁLISE DE QUALIDADE DAS CLASSIFICAÇÕES POR EMPRESA")
print("=" * 80)

from pyspark.sql.functions import (
    col, count, sum as spark_sum, when, lit, abs as spark_abs,
    round as spark_round, coalesce
)

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'
ANO_REFERENCIA = 2024

print(f"\n⚙️  Configurações:")
print(f"   Database ML: {DATABASE}")
print(f"   Database ECD: {DATABASE_ECD}")
print(f"   Ano: {ANO_REFERENCIA}")

# ================================================================================
# CARREGAR PREDIÇÕES
# ================================================================================

print(f"\n📥 Carregando predições...")
df_pred = spark.table(f"{DATABASE}.ecd_ml_predictions_ALL")
print(f"✅ Predições: {df_pred.count():,} registros")

# ================================================================================
# CARREGAR VALORES
# ================================================================================

print(f"\n📥 Carregando valores do ECD...")

df_balanco_valores = spark.sql(f"""
    SELECT 
        bp.id_ecd,
        bp.cod_agl as cd_conta,
        bp.vl_cta_fin as vl_conta_final,
        bp.ind_dc_cta_fin as ind_dc_final
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
""")

df_dre_valores = spark.sql(f"""
    SELECT 
        dre.id_ecd,
        dre.cod_agl as cd_conta,
        dre.vl_cta_fin as vl_conta_final,
        dre.ind_dc_cta_fin as ind_dc_final
    FROM {DATABASE_ECD}.ecd_rj150_demonstracao_resultado_exercicio dre
""")

print(f"✅ Balanço: {df_balanco_valores.count():,}")
print(f"✅ DRE: {df_dre_valores.count():,}")

# ================================================================================
# JOIN
# ================================================================================

print(f"\n🔗 JOIN...")

df_balanco_completo = df_pred.join(df_balanco_valores, on=['id_ecd', 'cd_conta'], how='inner')
df_dre_completo = df_pred.join(df_dre_valores, on=['id_ecd', 'cd_conta'], how='inner')

print(f"✅ Balanço: {df_balanco_completo.count():,}")
print(f"✅ DRE: {df_dre_completo.count():,}")

# ================================================================================
# 1. ESTATÍSTICAS (SEM LABEL - NÃO HÁ VALIDAÇÃO)
# ================================================================================

print(f"\n📊 ESTATÍSTICAS...")

stats_classificacao = df_balanco_completo.groupBy('cnpj', 'id_ecd').agg(
    count('*').alias('total_contas')
).withColumn('ano_calendario', lit(ANO_REFERENCIA)) \
 .withColumn('contas_corretas', col('total_contas')) \
 .withColumn('perc_corretas', lit(100.0))

stats_classificacao.createOrReplaceTempView("temp_stats")
spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_stats_classificacao_empresa")
spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_stats_classificacao_empresa
    STORED AS PARQUET  
    AS SELECT * FROM temp_stats
""")

print(f"✅ Salvo: {DATABASE}.ecd_ml_stats_classificacao_empresa")

# ================================================================================
# 2. VALORES DO BALANÇO
# ================================================================================

print(f"\n💰 CALCULANDO BALANÇO...")

df_balanco_ajustado = df_balanco_completo.withColumn(
    'vl_ajustado',
    when(col('ind_dc_final') == 'D', col('vl_conta_final'))
    .when(col('ind_dc_final') == 'C', -col('vl_conta_final'))
    .otherwise(col('vl_conta_final'))
)

df_balanco_classificado = df_balanco_ajustado.withColumn(
    'grupo_balanco',
    when(col('classificacao_nivel2').isin([
        'ATIVO', 'ATIVO_CIRCULANTE', 'ATIVO_NAO_CIRCULANTE',
        'DISPONIBILIDADES', 'CREDITOS', 'ESTOQUES',
        'IMOBILIZADO', 'INTANGIVEL', 'INVESTIMENTOS'
    ]), 'ATIVO')
    .when(col('classificacao_nivel2').isin([
        'PATRIMONIO_LIQUIDO', 'CAPITAL_SOCIAL',
        'RESERVAS', 'LUCROS_PREJUIZOS_ACUMULADOS'
    ]), 'PATRIMONIO_LIQUIDO')
    .when(col('classificacao_nivel2').isin([
        'PASSIVO', 'PASSIVO_CIRCULANTE', 'PASSIVO_NAO_CIRCULANTE',
        'EMPRESTIMOS_CP', 'FORNECEDORES',
        'SALARIOS_A_PAGAR', 'TRIBUTOS_A_PAGAR'
    ]), 'PASSIVO')
    .otherwise('OUTROS')
)

print(f"\n🔍 Distribuição:")
df_balanco_classificado.groupBy('grupo_balanco').count().show()

valores_balanco = df_balanco_classificado.filter(
    col('grupo_balanco').isin(['ATIVO', 'PASSIVO', 'PATRIMONIO_LIQUIDO'])
).groupBy('id_ecd', 'grupo_balanco').agg(
    spark_sum('vl_ajustado').alias('vl_saldo_final')
)

valores_pivot = valores_balanco.groupBy('id_ecd').pivot(
    'grupo_balanco', ['ATIVO', 'PASSIVO', 'PATRIMONIO_LIQUIDO']
).agg(spark_sum('vl_saldo_final'))

df_empresas = df_pred.select('id_ecd', 'cnpj').distinct() \
    .withColumn('ano_calendario', lit(ANO_REFERENCIA))

valores_com_cnpj = valores_pivot.join(df_empresas, on='id_ecd', how='inner')

valores_com_equacao = valores_com_cnpj.withColumn(
    'ativo', coalesce(col('ATIVO'), lit(0))
).withColumn(
    'passivo', coalesce(col('PASSIVO'), lit(0))
).withColumn(
    'patrimonio_liquido', coalesce(col('PATRIMONIO_LIQUIDO'), lit(0))
).withColumn(
    'equacao_contabil', col('ativo') - (col('passivo') + col('patrimonio_liquido'))
).withColumn(
    'equacao_contabil_perc',
    when(spark_abs(col('ativo')) > 0, 
         spark_round(spark_abs(col('equacao_contabil')) / spark_abs(col('ativo')) * 100, 4)
    ).otherwise(0)
).withColumn(
    'equacao_valida', when(col('equacao_contabil_perc') <= 10.0, 'SIM').otherwise('NAO')
)

valores_com_equacao.createOrReplaceTempView("temp_balanco")
spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_valores_balanco_empresa")
spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_valores_balanco_empresa
    STORED AS PARQUET
    AS SELECT * FROM temp_balanco
""")

print(f"✅ Salvo: {DATABASE}.ecd_ml_valores_balanco_empresa")

print(f"\n⚖️  Erro na equação:")
valores_com_equacao.groupBy(
    when(col('equacao_contabil_perc') <= 0.5, '≤ 0.5%')
    .when(col('equacao_contabil_perc') <= 1.0, '0.5-1%')
    .when(col('equacao_contabil_perc') <= 5.0, '1-5%')
    .when(col('equacao_contabil_perc') <= 10.0, '5-10%')
    .otherwise('> 10%').alias('faixa')
).count().orderBy('faixa').show()

# ================================================================================
# 3. DRE
# ================================================================================

print(f"\n📈 CALCULANDO DRE...")

df_dre_ajustado = df_dre_completo.withColumn(
    'vl_ajustado',
    when(col('ind_dc_final') == 'D', col('vl_conta_final'))
    .when(col('ind_dc_final') == 'C', -col('vl_conta_final'))
    .otherwise(col('vl_conta_final'))
)

df_dre_classificado = df_dre_ajustado.withColumn(
    'grupo_dre',
    when(col('classificacao_nivel2').isin([
        'RECEITA_BRUTA', 'DEDUCOES_RECEITA', 'RECEITAS_FINANCEIRAS'
    ]), 'RECEITA')
    .when(col('classificacao_nivel2').isin([
        'CUSTOS', 'DESPESAS_OPERACIONAIS', 'DESPESAS_FINANCEIRAS',
        'IMPOSTOS_RESULTADO', 'OUTRAS_RECEITAS_DESPESAS'
    ]), 'DESPESA')
    .otherwise('OUTROS')
)

valores_dre = df_dre_classificado.filter(
    col('grupo_dre').isin(['RECEITA', 'DESPESA'])
).groupBy('id_ecd', 'grupo_dre').agg(
    spark_sum('vl_ajustado').alias('valor')
)

valores_dre_pivot = valores_dre.groupBy('id_ecd').pivot(
    'grupo_dre', ['RECEITA', 'DESPESA']
).agg(spark_sum('valor'))

valores_dre_final = valores_dre_pivot.join(df_empresas, on='id_ecd', how='inner') \
    .withColumn('receitas', spark_abs(coalesce(col('RECEITA'), lit(0)))) \
    .withColumn('despesas', spark_abs(coalesce(col('DESPESA'), lit(0)))) \
    .withColumn('resultado', col('receitas') - col('despesas')) \
    .select('id_ecd', 'cnpj', 'ano_calendario', 'receitas', 'despesas', 'resultado')

valores_dre_final.createOrReplaceTempView("temp_dre")
spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_valores_dre_empresa")
spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_valores_dre_empresa
    STORED AS PARQUET
    AS SELECT * FROM temp_dre
""")

print(f"✅ Salvo: {DATABASE}.ecd_ml_valores_dre_empresa")

# ================================================================================
# 4. CONSOLIDAR
# ================================================================================

print(f"\n🔗 CONSOLIDANDO...")

df_final = stats_classificacao \
    .join(valores_com_equacao.drop('ano_calendario'), on=['cnpj', 'id_ecd'], how='left') \
    .join(valores_dre_final.drop('ano_calendario'), on=['cnpj', 'id_ecd'], how='left') \
    .withColumn('classificacao_boa', lit('SIM')) \
    .withColumn('dados_completos', 
        when((col('ativo').isNotNull()) & (col('passivo').isNotNull()) & 
             (col('patrimonio_liquido').isNotNull()) & (col('receitas').isNotNull()) & 
             (col('despesas').isNotNull()), 'SIM').otherwise('NAO')) \
    .withColumn('apta_indicadores',
        when((col('equacao_valida') == 'SIM') & 
             (col('dados_completos') == 'SIM') & (spark_abs(col('ativo')) > 10000), 
             'SIM').otherwise('NAO'))

df_final.createOrReplaceTempView("temp_consolidado")
spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_empresas_consolidado")
spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_empresas_consolidado
    STORED AS PARQUET
    AS SELECT * FROM temp_consolidado
""")

print(f"✅ Salvo: {DATABASE}.ecd_ml_empresas_consolidado")

# ================================================================================
# 5. ESTATÍSTICAS FINAIS
# ================================================================================

print(f"\n📊 ESTATÍSTICAS FINAIS")
print("=" * 80)

total = df_final.count()
aptas = df_final.filter(col('apta_indicadores') == 'SIM').count()

print(f"\n✅ Total: {total:,} empresas")
print(f"   ✅ Aptas: {aptas:,} ({aptas/total*100:.2f}%)")
print(f"   ❌ Não aptas: {total-aptas:,}")

print(f"\n📋 Motivos:")
print(f"   - Equação > 10%: {df_final.filter(col('equacao_valida') == 'NAO').count():,}")
print(f"   - Dados incompletos: {df_final.filter(col('dados_completos') == 'NAO').count():,}")

if aptas > 0:
    print(f"\n🏆 TOP 20 APTAS:")
    df_final.filter(col('apta_indicadores') == 'SIM') \
        .select('cnpj', 'total_contas', 'equacao_contabil_perc', 
                'ativo', 'patrimonio_liquido', 'resultado') \
        .orderBy(col('equacao_contabil_perc').asc()) \
        .limit(20) \
        .show(truncate=False)

print("\n" + "=" * 80)
print("✅ SCRIPT 1 CONCLUÍDO!")
print("=" * 80)

🔍 ANÁLISE DE QUALIDADE DAS CLASSIFICAÇÕES POR EMPRESA

⚙️  Configurações:
   Database ML: neac
   Database ECD: usr_sat_ecd
   Ano: 2024

📥 Carregando predições...
✅ Predições: 144,602,905 registros

📥 Carregando valores do ECD...
✅ Balanço: 144,602,899
✅ DRE: 40,260,048

🔗 JOIN...


✅ Balanço: 460,360,611


✅ DRE: 106,068

📊 ESTATÍSTICAS...


✅ Salvo: neac.ecd_ml_stats_classificacao_empresa

💰 CALCULANDO BALANÇO...

🔍 Distribuição:


+------------------+---------+
|     grupo_balanco|    count|
+------------------+---------+
|           PASSIVO|213044367|
|            OUTROS|   172419|
|PATRIMONIO_LIQUIDO|  3484536|
|             ATIVO|243659289|
+------------------+---------+



...


✅ Total: 384,493 empresas
   ✅ Aptas: 0 (0.00%)
   ❌ Não aptas: 384,493

📋 Motivos:


                                                                                ]

   - Equação > 10%: 380,999


[Stage 2809:==============================================>       (25 + 4) / 29]]

   - Dados incompletos: 384,490

✅ SCRIPT 1 CONCLUÍDO!


In [36]:
# ================================================================================
# SCRIPT 1.1: ANÁLISE DE TOLERÂNCIAS NA EQUAÇÃO CONTÁBIL
# ================================================================================

print("📊 ANÁLISE DE TOLERÂNCIAS NA EQUAÇÃO CONTÁBIL")
print("=" * 80)

from pyspark.sql.functions import (
    col, when, count, sum as spark_sum, 
    round as spark_round, abs as spark_abs
)

# ================================================================================
# CONFIGURAÇÕES
# ================================================================================

DATABASE = 'neac'
ANO_REFERENCIA = 2024

# Tolerâncias a testar
TOLERANCIAS = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 15.0, 20.0]

print(f"\n⚙️  Configurações:")
print(f"   Database: {DATABASE}")
print(f"   Ano: {ANO_REFERENCIA}")
print(f"   Tolerâncias a testar: {TOLERANCIAS}")

# ================================================================================
# CARREGAR DADOS
# ================================================================================

print(f"\n📥 Carregando dados consolidados...")

df_empresas = spark.table(f"{DATABASE}.ecd_ml_empresas_consolidado")
total_empresas = df_empresas.count()

print(f"✅ Total de empresas: {total_empresas:,}")

if total_empresas == 0:
    print("\n⚠️  Execute Script 1 primeiro!")
    raise SystemExit

# ================================================================================
# ANÁLISE POR TOLERÂNCIA
# ================================================================================

print(f"\n📊 TESTANDO DIFERENTES TOLERÂNCIAS")
print("=" * 80)

resultados = []

for tolerancia in TOLERANCIAS:
    # Filtrar empresas dentro da tolerância
    df_dentro_tolerancia = df_empresas.filter(
        (col('equacao_contabil_perc') <= tolerancia) &
        (col('ativo').isNotNull()) &
        (col('passivo').isNotNull()) &
        (col('patrimonio_liquido').isNotNull()) &
        (col('receitas').isNotNull()) &
        (col('despesas').isNotNull()) &
        (spark_abs(col('ativo')) > 10000)  # Ativo mínimo
    )
    
    total_aptas = df_dentro_tolerancia.count()
    perc_aptas = (total_aptas / total_empresas * 100) if total_empresas > 0 else 0
    
    # Calcular estatísticas do erro
    stats = df_dentro_tolerancia.select(
        spark_round(col('equacao_contabil_perc').cast('double'), 4).alias('erro')
    ).summary('mean', 'stddev', 'min', 'max').collect()
    
    erro_medio = float(stats[0]['erro']) if stats[0]['erro'] else 0
    erro_stddev = float(stats[1]['erro']) if stats[1]['erro'] else 0
    erro_min = float(stats[2]['erro']) if stats[2]['erro'] else 0
    erro_max = float(stats[3]['erro']) if stats[3]['erro'] else 0
    
    resultados.append({
        'tolerancia': tolerancia,
        'total_aptas': total_aptas,
        'perc_aptas': perc_aptas,
        'erro_medio': erro_medio,
        'erro_stddev': erro_stddev,
        'erro_min': erro_min,
        'erro_max': erro_max
    })
    
    print(f"\n📊 Tolerância: {tolerancia}%")
    print(f"   Empresas aptas: {total_aptas:,} ({perc_aptas:.2f}%)")
    print(f"   Erro médio: {erro_medio:.4f}%")
    print(f"   Desvio padrão: {erro_stddev:.4f}%")
    print(f"   Erro mín/máx: {erro_min:.4f}% / {erro_max:.4f}%")

# ================================================================================
# RESUMO COMPARATIVO
# ================================================================================

print(f"\n📊 RESUMO COMPARATIVO")
print("=" * 80)
print(f"{'Tolerância':<12} {'Empresas':>12} {'% Total':>10} {'Erro Médio':>12} {'Desvio':>10}")
print("-" * 80)

for r in resultados:
    print(f"{r['tolerancia']:>10.1f}%  {r['total_aptas']:>12,}  {r['perc_aptas']:>9.2f}%  {r['erro_medio']:>11.4f}%  {r['erro_stddev']:>9.4f}%")

# ================================================================================
# RECOMENDAÇÃO
# ================================================================================

print(f"\n💡 RECOMENDAÇÃO")
print("=" * 80)

# Encontrar tolerância que atinge pelo menos 10% das empresas
for r in resultados:
    if r['perc_aptas'] >= 10.0:
        print(f"\n✅ Tolerância recomendada: {r['tolerancia']}%")
        print(f"   - Captura: {r['total_aptas']:,} empresas ({r['perc_aptas']:.2f}%)")
        print(f"   - Erro médio: {r['erro_medio']:.4f}%")
        print(f"   - Qualidade: {'ALTA' if r['erro_medio'] < 1.0 else 'MÉDIA' if r['erro_medio'] < 5.0 else 'BAIXA'}")
        break
else:
    print(f"\n⚠️  Nenhuma tolerância atinge 10% das empresas")
    print(f"   Use a maior tolerância disponível: {TOLERANCIAS[-1]}%")

# ================================================================================
# DISTRIBUIÇÃO DO ERRO
# ================================================================================

print(f"\n📊 DISTRIBUIÇÃO DO ERRO (TODAS AS EMPRESAS)")
print("=" * 80)

df_empresas.groupBy(
    when(col('equacao_contabil_perc') <= 0.1, '≤ 0.1%')
    .when(col('equacao_contabil_perc') <= 0.5, '0.1-0.5%')
    .when(col('equacao_contabil_perc') <= 1.0, '0.5-1.0%')
    .when(col('equacao_contabil_perc') <= 2.0, '1.0-2.0%')
    .when(col('equacao_contabil_perc') <= 5.0, '2.0-5.0%')
    .when(col('equacao_contabil_perc') <= 10.0, '5.0-10.0%')
    .when(col('equacao_contabil_perc') <= 20.0, '10.0-20.0%')
    .otherwise('> 20.0%')
    .alias('faixa_erro')
).count().orderBy('faixa_erro').show(truncate=False)

# ================================================================================
# ANÁLISE DE EMPRESAS COM ERRO ZERO
# ================================================================================

print(f"\n🎯 EMPRESAS COM EQUAÇÃO CONTÁBIL PERFEITA")
print("=" * 80)

df_perfeitas = df_empresas.filter(
    (col('equacao_contabil_perc') <= 0.1) &
    (spark_abs(col('ativo')) > 10000)
)

total_perfeitas = df_perfeitas.count()
perc_perfeitas = (total_perfeitas / total_empresas * 100) if total_empresas > 0 else 0

print(f"\n✅ Empresas com erro ≤ 0.1%: {total_perfeitas:,} ({perc_perfeitas:.2f}%)")

if total_perfeitas > 0:
    print(f"\n📋 Top 10 empresas perfeitas:")
    df_perfeitas.select(
        'cnpj',
        'total_contas',
        spark_round('equacao_contabil_perc', 6).alias('erro_%'),
        spark_round('ativo', 2).alias('ativo'),
        spark_round('passivo', 2).alias('passivo'),
        spark_round('patrimonio_liquido', 2).alias('pl')
    ).orderBy(col('equacao_contabil_perc').asc()) \
     .limit(10) \
     .show(truncate=False)

# ================================================================================
# ANÁLISE DE EMPRESAS COM DADOS COMPLETOS
# ================================================================================

print(f"\n📊 EMPRESAS COM DADOS COMPLETOS")
print("=" * 80)

df_completas = df_empresas.filter(
    (col('ativo').isNotNull()) &
    (col('passivo').isNotNull()) &
    (col('patrimonio_liquido').isNotNull()) &
    (col('receitas').isNotNull()) &
    (col('despesas').isNotNull())
)

total_completas = df_completas.count()
perc_completas = (total_completas / total_empresas * 100) if total_empresas > 0 else 0

print(f"\n✅ Empresas com dados completos: {total_completas:,} ({perc_completas:.2f}%)")

# Dentro dessas, quantas têm boa equação?
df_completas_boas = df_completas.filter(col('equacao_contabil_perc') <= 10.0)
total_completas_boas = df_completas_boas.count()
perc_completas_boas = (total_completas_boas / total_completas * 100) if total_completas > 0 else 0

print(f"✅ Com dados completos E equação ≤ 10%: {total_completas_boas:,} ({perc_completas_boas:.2f}%)")

print("\n" + "=" * 80)
print("✅ SCRIPT 1.1 CONCLUÍDO!")
print("=" * 80)

📊 ANÁLISE DE TOLERÂNCIAS NA EQUAÇÃO CONTÁBIL

⚙️  Configurações:
   Database: neac
   Ano: 2024
   Tolerâncias a testar: [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 15.0, 20.0]

📥 Carregando dados consolidados...
✅ Total de empresas: 384,493

📊 TESTANDO DIFERENTES TOLERÂNCIAS

📊 Tolerância: 0.1%
   Empresas aptas: 0 (0.00%)
   Erro médio: 0.0000%
   Desvio padrão: 0.0000%
   Erro mín/máx: 0.0000% / 0.0000%

📊 Tolerância: 0.5%
   Empresas aptas: 0 (0.00%)
   Erro médio: 0.0000%
   Desvio padrão: 0.0000%
   Erro mín/máx: 0.0000% / 0.0000%

📊 Tolerância: 1.0%
   Empresas aptas: 0 (0.00%)
   Erro médio: 0.0000%
   Desvio padrão: 0.0000%
   Erro mín/máx: 0.0000% / 0.0000%

📊 Tolerância: 2.0%
   Empresas aptas: 0 (0.00%)
   Erro médio: 0.0000%
   Desvio padrão: 0.0000%
   Erro mín/máx: 0.0000% / 0.0000%

📊 Tolerância: 5.0%
   Empresas aptas: 0 (0.00%)
   Erro médio: 0.0000%
   Desvio padrão: 0.0000%
   Erro mín/máx: 0.0000% / 0.0000%

📊 Tolerância: 10.0%
   Empresas aptas: 0 (0.00%)
   Erro médio: 0.000

In [37]:
# ================================================================================
# SCRIPT 1.2: IDENTIFICAR EMPRESAS CANDIDATAS PARA AJUSTE MANUAL
# ================================================================================

print("🔍 IDENTIFICAR EMPRESAS CANDIDATAS PARA AJUSTE MANUAL")
print("=" * 80)

from pyspark.sql.functions import (
    col, when, count, sum as spark_sum, 
    round as spark_round, abs as spark_abs,
    desc, asc
)

# ================================================================================
# CONFIGURAÇÕES
# ================================================================================

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'
ANO_REFERENCIA = 2024

# Critérios para candidatas a ajuste
ERRO_MIN = 1.0   # Erro mínimo: 1% (abaixo disso já está bom)
ERRO_MAX = 15.0  # Erro máximo: 15% (acima disso, muito ruim)
MIN_ATIVO = 100000  # Ativo mínimo para considerar
TOP_N = 50  # Top N empresas para analisar

print(f"\n⚙️  Critérios:")
print(f"   Erro mínimo: {ERRO_MIN}%")
print(f"   Erro máximo: {ERRO_MAX}%")
print(f"   Ativo mínimo: R$ {MIN_ATIVO:,.2f}")
print(f"   Top N: {TOP_N}")

# ================================================================================
# CARREGAR DADOS
# ================================================================================

print(f"\n📥 Carregando dados...")

df_empresas = spark.table(f"{DATABASE}.ecd_ml_empresas_consolidado")
total_empresas = df_empresas.count()

print(f"✅ Total: {total_empresas:,} empresas")

if total_empresas == 0:
    print("\n⚠️  Execute Script 1 primeiro!")
    raise SystemExit

# ================================================================================
# FILTRAR CANDIDATAS
# ================================================================================

print(f"\n🔍 Filtrando candidatas para ajuste...")

df_candidatas = df_empresas.filter(
    (col('equacao_contabil_perc') > ERRO_MIN) &
    (col('equacao_contabil_perc') <= ERRO_MAX) &
    (spark_abs(col('ativo')) >= MIN_ATIVO) &
    (col('ativo').isNotNull()) &
    (col('passivo').isNotNull()) &
    (col('patrimonio_liquido').isNotNull())
)

total_candidatas = df_candidatas.count()
perc_candidatas = (total_candidatas / total_empresas * 100) if total_empresas > 0 else 0

print(f"✅ Candidatas: {total_candidatas:,} ({perc_candidatas:.2f}%)")

if total_candidatas == 0:
    print("\n⚠️  Nenhuma candidata encontrada com esses critérios!")
    print(f"   Tente ajustar ERRO_MIN, ERRO_MAX ou MIN_ATIVO")
    raise SystemExit

# ================================================================================
# PRIORIZAR POR CRITÉRIOS
# ================================================================================

print(f"\n📊 PRIORIZANDO CANDIDATAS...")
print("-" * 80)

# Adicionar score de prioridade
df_priorizadas = df_candidatas.withColumn(
    'score_prioridade',
    # Quanto menor o erro, maior a prioridade (mais fácil de corrigir)
    # Quanto maior o ativo, maior a prioridade (mais importante)
    (1 / col('equacao_contabil_perc')) * spark_abs(col('ativo'))
).withColumn(
    'potencial_correcao',
    when(col('equacao_contabil_perc') <= 2.0, 'ALTO')
    .when(col('equacao_contabil_perc') <= 5.0, 'MEDIO')
    .otherwise('BAIXO')
)

# ================================================================================
# TOP CANDIDATAS
# ================================================================================

print(f"\n🏆 TOP {TOP_N} CANDIDATAS PARA AJUSTE MANUAL")
print("=" * 80)

df_top = df_priorizadas.select(
    'cnpj',
    'id_ecd',
    'total_contas',
    spark_round('equacao_contabil_perc', 4).alias('erro_%'),
    spark_round('equacao_contabil', 2).alias('erro_abs'),
    spark_round('ativo', 2).alias('ativo'),
    spark_round('passivo', 2).alias('passivo'),
    spark_round('patrimonio_liquido', 2).alias('pl'),
    'potencial_correcao'
).orderBy(col('score_prioridade').desc())

df_top.limit(TOP_N).show(truncate=False)

# ================================================================================
# SALVAR LISTA DE CANDIDATAS
# ================================================================================

print(f"\n💾 Salvando lista de candidatas...")

df_top.createOrReplaceTempView("temp_candidatas_ajuste")

spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_candidatas_ajuste_manual")

spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_candidatas_ajuste_manual
    STORED AS PARQUET
    AS SELECT * FROM temp_candidatas_ajuste
""")

print(f"✅ Salvo: {DATABASE}.ecd_ml_candidatas_ajuste_manual")

# ================================================================================
# ANÁLISE DETALHADA DE UMA EMPRESA EXEMPLO
# ================================================================================

print(f"\n🔬 ANÁLISE DETALHADA DA EMPRESA #1")
print("=" * 80)

# Pegar primeira empresa
empresa_exemplo = df_top.first()

if empresa_exemplo:
    id_ecd_exemplo = empresa_exemplo['id_ecd']
    cnpj_exemplo = empresa_exemplo['cnpj']
    
    print(f"\n📋 Empresa: {cnpj_exemplo}")
    print(f"   ID ECD: {id_ecd_exemplo}")
    print(f"   Erro: {empresa_exemplo['erro_%']:.4f}%")
    print(f"   Ativo: R$ {empresa_exemplo['ativo']:,.2f}")
    print(f"   Passivo: R$ {empresa_exemplo['passivo']:,.2f}")
    print(f"   PL: R$ {empresa_exemplo['pl']:,.2f}")
    
    # Buscar detalhes das contas
    print(f"\n📊 Distribuição de contas por grupo:")
    
    df_contas_empresa = spark.sql(f"""
        SELECT 
            pred.classificacao_nivel2,
            COUNT(*) as qtd_contas,
            ROUND(SUM(
                CASE 
                    WHEN bp.ind_dc_cta_fin = 'D' THEN bp.vl_cta_fin
                    WHEN bp.ind_dc_cta_fin = 'C' THEN -bp.vl_cta_fin
                    ELSE bp.vl_cta_fin
                END
            ), 2) as valor_total
        FROM {DATABASE}.ecd_ml_predictions_ALL pred
        INNER JOIN {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
            ON pred.id_ecd = bp.id_ecd
            AND pred.cd_conta = bp.cod_agl
        WHERE pred.id_ecd = {id_ecd_exemplo}
        GROUP BY pred.classificacao_nivel2
        ORDER BY valor_total DESC
    """)
    
    df_contas_empresa.show(30, truncate=False)
    
    # Identificar possíveis problemas
    print(f"\n🔍 POSSÍVEIS PROBLEMAS:")
    
    # Verificar se há grupos com valores muito pequenos
    df_problemas = df_contas_empresa.filter(
        (spark_abs(col('valor_total')) > 0) & 
        (spark_abs(col('valor_total')) < 1000)
    )
    
    if df_problemas.count() > 0:
        print(f"\n⚠️  Grupos com valores pequenos (< R$ 1.000):")
        df_problemas.show(truncate=False)
    
    # Verificar se há classificações estranhas
    df_outros = df_contas_empresa.filter(col('classificacao_nivel2') == 'OUTROS')
    
    if df_outros.count() > 0:
        print(f"\n⚠️  Contas classificadas como 'OUTROS':")
        df_outros.show(truncate=False)

# ================================================================================
# ESTATÍSTICAS DAS CANDIDATAS
# ================================================================================

print(f"\n📊 ESTATÍSTICAS DAS CANDIDATAS")
print("=" * 80)

print(f"\n📈 Distribuição por potencial de correção:")
df_priorizadas.groupBy('potencial_correcao').count() \
    .orderBy('potencial_correcao').show()

print(f"\n📈 Distribuição por faixa de erro:")
df_priorizadas.groupBy(
    when(col('equacao_contabil_perc') <= 2.0, '1-2%')
    .when(col('equacao_contabil_perc') <= 5.0, '2-5%')
    .when(col('equacao_contabil_perc') <= 10.0, '5-10%')
    .otherwise('10-15%')
    .alias('faixa_erro')
).count().orderBy('faixa_erro').show()

print(f"\n📈 Estatísticas de ativo das candidatas:")
df_priorizadas.select('ativo').summary(
    'count', 'mean', 'stddev', 'min', '25%', '50%', '75%', 'max'
).show(truncate=False)

# ================================================================================
# RECOMENDAÇÕES
# ================================================================================

print(f"\n💡 RECOMENDAÇÕES")
print("=" * 80)

print(f"\n1. 🎯 PRIORIZE AS TOP {min(10, TOP_N)} EMPRESAS")
print(f"   - Maior potencial de correção")
print(f"   - Maior ativo (impacto)")

print(f"\n2. 🔍 INVESTIGUE:")
print(f"   - Contas classificadas como 'OUTROS'")
print(f"   - Grupos com valores muito pequenos")
print(f"   - Contas com nomes genéricos")

print(f"\n3. ✏️ AJUSTES POSSÍVEIS:")
print(f"   - Reclassificar manualmente algumas contas")
print(f"   - Criar regras específicas para padrões identificados")
print(f"   - Retreinar modelo com exemplos corrigidos")

print(f"\n4. 📊 VALIDAÇÃO:")
print(f"   - Após ajustes, recalcular equação")
print(f"   - Verificar se erro caiu abaixo de {ERRO_MIN}%")

print("\n" + "=" * 80)
print("✅ SCRIPT 1.2 CONCLUÍDO!")
print("=" * 80)

print(f"\n📋 Próximos passos:")
print(f"   1. Analise a tabela: {DATABASE}.ecd_ml_candidatas_ajuste_manual")
print(f"   2. Escolha empresas para ajuste manual")
print(f"   3. Use Script 2 e 3 com tolerância ajustada")

🔍 IDENTIFICAR EMPRESAS CANDIDATAS PARA AJUSTE MANUAL

⚙️  Critérios:
   Erro mínimo: 1.0%
   Erro máximo: 15.0%
   Ativo mínimo: R$ 100,000.00
   Top N: 50

📥 Carregando dados...
✅ Total: 384,493 empresas

🔍 Filtrando candidatas para ajuste...
✅ Candidatas: 52 (0.01%)

📊 PRIORIZANDO CANDIDATAS...
--------------------------------------------------------------------------------

🏆 TOP 50 CANDIDATAS PARA AJUSTE MANUAL
+------+------+------------+-------+-----------+------------+------------+------------+------------------+
|cnpj  |id_ecd|total_contas|erro_% |erro_abs   |ativo       |passivo     |pl          |potencial_correcao|
+------+------+------------+-------+-----------+------------+------------+------------+------------------+
|403178|403178|68          |2.0265 |243207.86  |12001247.11 |12400289.25 |-642250.00  |MEDIO             |
|298154|298154|54          |8.4766 |-2886893.36|34057245.60 |37231459.96 |-287321.00  |BAIXO             |
|328190|328190|203         |4.9084 |-927708.68

+--------------------+----------+-----------+
|classificacao_nivel2|qtd_contas|valor_total|
+--------------------+----------+-----------+
|PASSIVO_CIRCULANTE  |31        |14517841.88|
|ATIVO_CIRCULANTE    |21        |8442357.52 |
|ATIVO               |3         |3465556.63 |
|DISPONIBILIDADES    |3         |60411.53   |
|IMOBILIZADO         |1         |32921.43   |
|PASSIVO             |7         |-423507.68 |
|CAPITAL_SOCIAL      |1         |-642250.00 |
|FORNECEDORES        |1         |-1694044.95|
+--------------------+----------+-----------+


🔍 POSSÍVEIS PROBLEMAS:



📊 ESTATÍSTICAS DAS CANDIDATAS

📈 Distribuição por potencial de correção:


+------------------+-----+
|potencial_correcao|count|
+------------------+-----+
|              ALTO|    2|
|             BAIXO|   45|
|             MEDIO|    5|
+------------------+-----+


📈 Distribuição por faixa de erro:
+----------+-----+
|faixa_erro|count|
+----------+-----+
|      1-2%|    2|
|    10-15%|   24|
|      2-5%|    5|
|     5-10%|   21|
+----------+-----+


📈 Estatísticas de ativo das candidatas:
+-------+-----------------+
|summary|ativo            |
+-------+-----------------+
|count  |52               |
|mean   |3587705.966346   |
|stddev |8463930.069514671|
|min    |-18900473.69     |
|25%    |137773.84        |
|50%    |412130.93        |
|75%    |2988876.64       |
|max    |34057245.60      |
+-------+-----------------+


💡 RECOMENDAÇÕES


TypeError: min() takes 1 positional argument but 2 were given

In [38]:
# ================================================================================
# SCRIPT 2: EMPRESAS APTAS E CÁLCULO DE INDICADORES FINANCEIROS
# ================================================================================

print("💼 EMPRESAS APTAS PARA CÁLCULO DE INDICADORES FINANCEIROS")
print("=" * 80)

from pyspark.sql.functions import (
    col, when, lit, round as spark_round, abs as spark_abs,
    coalesce, count, sum as spark_sum, avg
)
import pandas as pd

# ================================================================================
# CONFIGURAÇÕES
# ================================================================================

DATABASE = 'neac'
ANO_REFERENCIA = 2024

# Critérios de aptidão (AJUSTÁVEIS)
MAX_ERRO_EQUACAO = 10.0      # Máximo 10% de erro na equação contábil
MIN_ATIVO = 10000            # Ativo mínimo

print(f"\n⚙️  Critérios de aptidão:")
print(f"   - Erro máximo equação: {MAX_ERRO_EQUACAO}%")
print(f"   - Ativo mínimo: R$ {MIN_ATIVO:,.2f}")

# ================================================================================
# CARREGAR DADOS CONSOLIDADOS
# ================================================================================

print(f"\n📥 Carregando dados consolidados...")

df_empresas = spark.table(f"{DATABASE}.ecd_ml_empresas_consolidado")
total_empresas = df_empresas.count()

print(f"✅ {total_empresas:,} empresas carregadas")

# ================================================================================
# FILTRAR EMPRESAS APTAS
# ================================================================================

print(f"\n🔍 Filtrando empresas aptas...")

df_aptas = df_empresas.filter(
    (col('equacao_contabil_perc') <= MAX_ERRO_EQUACAO) &
    (col('ativo').isNotNull()) &
    (col('passivo').isNotNull()) &
    (col('patrimonio_liquido').isNotNull()) &
    (col('receitas').isNotNull()) &
    (col('despesas').isNotNull()) &
    (spark_abs(col('ativo')) >= MIN_ATIVO)
)

total_aptas = df_aptas.count()
perc_aptas = (total_aptas / total_empresas * 100) if total_empresas > 0 else 0

print(f"✅ {total_aptas:,} empresas aptas ({perc_aptas:.2f}% do total)")

if total_aptas == 0:
    print("\n" + "=" * 80)
    print("⚠️  NENHUMA EMPRESA APTA ENCONTRADA!")
    print("=" * 80)
    print("\n💡 Sugestões:")
    print(f"   1. Aumentar MAX_ERRO_EQUACAO (atual: {MAX_ERRO_EQUACAO}%)")
    print(f"   2. Reduzir MIN_ATIVO (atual: R$ {MIN_ATIVO:,.2f})")
    
else:
    # ============================================================================
    # ANÁLISE PRELIMINAR
    # ============================================================================
    
    print(f"\n📊 ANÁLISE PRELIMINAR")
    print("-" * 80)
    
    # Distribuição de erro
    print(f"\n⚖️  Distribuição de erro na equação:")
    df_aptas.groupBy(
        when(col('equacao_contabil_perc') <= 1.0, '≤ 1%')
        .when(col('equacao_contabil_perc') <= 2.0, '1-2%')
        .when(col('equacao_contabil_perc') <= 5.0, '2-5%')
        .otherwise('5-10%')
        .alias('faixa_erro')
    ).count().orderBy('faixa_erro').show()
    
    # Estatísticas dos valores
    print(f"\n💰 Estatísticas dos valores contábeis:")
    df_aptas.select(
        spark_round(avg('ativo'), 2).alias('ativo_medio'),
        spark_round(avg('passivo'), 2).alias('passivo_medio'),
        spark_round(avg('patrimonio_liquido'), 2).alias('pl_medio'),
        spark_round(avg('receitas'), 2).alias('receitas_media'),
        spark_round(avg('resultado'), 2).alias('resultado_medio')
    ).show(truncate=False)
    
    # ============================================================================
    # CALCULAR INDICADORES FINANCEIROS
    # ============================================================================
    
    print(f"\n📊 CALCULANDO INDICADORES FINANCEIROS...")
    print("-" * 80)
    
    df_indicadores = df_aptas.withColumn(
        # ========================================================================
        # LIQUIDEZ
        # ========================================================================
        'liquidez_corrente',
        when(col('passivo') != 0,
             spark_round(col('ativo') / col('passivo'), 4)
        ).otherwise(None)
    ).withColumn(
        'liquidez_geral',
        when(col('passivo') != 0,
             spark_round(col('ativo') / col('passivo'), 4)
        ).otherwise(None)
    ).withColumn(
        # ========================================================================
        # ENDIVIDAMENTO
        # ========================================================================
        'endividamento_geral',
        when(col('ativo') != 0,
             spark_round((col('passivo') / col('ativo')) * 100, 2)
        ).otherwise(None)
    ).withColumn(
        'participacao_capital_terceiros',
        when((col('patrimonio_liquido') != 0),
             spark_round((col('passivo') / col('patrimonio_liquido')) * 100, 2)
        ).otherwise(None)
    ).withColumn(
        'composicao_endividamento',
        when((col('passivo') + col('patrimonio_liquido')) != 0,
             spark_round(
                 (col('passivo') / (col('passivo') + col('patrimonio_liquido'))) * 100, 2
             )
        ).otherwise(None)
    ).withColumn(
        # ========================================================================
        # ESTRUTURA DE CAPITAL
        # ========================================================================
        'imobilizacao_pl',
        when(col('patrimonio_liquido') != 0,
             spark_round((col('ativo') / col('patrimonio_liquido')) * 100, 2)
        ).otherwise(None)
    ).withColumn(
        'imobilizacao_recursos_nao_correntes',
        when((col('patrimonio_liquido') + col('passivo')) != 0,
             spark_round(
                 (col('ativo') / (col('patrimonio_liquido') + col('passivo'))) * 100, 2
             )
        ).otherwise(None)
    ).withColumn(
        # ========================================================================
        # RENTABILIDADE
        # ========================================================================
        'margem_liquida',
        when(col('receitas') != 0,
             spark_round((col('resultado') / col('receitas')) * 100, 2)
        ).otherwise(None)
    ).withColumn(
        'rentabilidade_ativo',
        when(col('ativo') != 0,
             spark_round((col('resultado') / col('ativo')) * 100, 2)
        ).otherwise(None)
    ).withColumn(
        'rentabilidade_pl',
        when(col('patrimonio_liquido') != 0,
             spark_round((col('resultado') / col('patrimonio_liquido')) * 100, 2)
        ).otherwise(None)
    ).withColumn(
        # ========================================================================
        # GIRO
        # ========================================================================
        'giro_ativo',
        when(col('ativo') != 0,
             spark_round(col('receitas') / col('ativo'), 4)
        ).otherwise(None)
    )
    
    print("✅ Indicadores calculados!")
    
    indicadores_calculados = [
        'liquidez_corrente', 'liquidez_geral',
        'endividamento_geral', 'participacao_capital_terceiros', 'composicao_endividamento',
        'imobilizacao_pl', 'imobilizacao_recursos_nao_correntes',
        'margem_liquida', 'rentabilidade_ativo', 'rentabilidade_pl',
        'giro_ativo'
    ]
    
    # ============================================================================
    # SALVAR
    # ============================================================================
    
    print(f"\n💾 Salvando empresas aptas com indicadores...")
    
    df_indicadores.createOrReplaceTempView("temp_empresas_aptas_indicadores")
    
    spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_empresas_aptas_indicadores")
    
    spark.sql(f"""
        CREATE TABLE {DATABASE}.ecd_ml_empresas_aptas_indicadores
        STORED AS PARQUET
        AS SELECT * FROM temp_empresas_aptas_indicadores
    """)
    
    print(f"✅ Salvo: {DATABASE}.ecd_ml_empresas_aptas_indicadores")
    
    # ============================================================================
    # ESTATÍSTICAS
    # ============================================================================
    
    print(f"\n📊 ESTATÍSTICAS DOS INDICADORES")
    print("=" * 80)
    
    stats = df_indicadores.select(indicadores_calculados).summary(
        'count', 'mean', 'stddev', 'min', '25%', '50%', '75%', 'max'
    )
    
    stats.show(truncate=False)
    
    # ============================================================================
    # OUTLIERS
    # ============================================================================
    
    print(f"\n⚠️  ANÁLISE DE VALORES EXTREMOS")
    print("-" * 80)
    
    df_extremos = df_indicadores.filter(
        (spark_abs(col('endividamento_geral')) > 100) |
        (spark_abs(col('rentabilidade_pl')) > 200) |
        (spark_abs(col('rentabilidade_ativo')) > 100)
    )
    
    total_extremos = df_extremos.count()
    print(f"Empresas com valores extremos: {total_extremos:,} ({total_extremos/total_aptas*100:.2f}%)")
    
    # ============================================================================
    # TOP EMPRESAS
    # ============================================================================
    
    print(f"\n🏆 TOP 20 EMPRESAS (MENOR ERRO NA EQUAÇÃO)")
    print("-" * 80)
    
    df_indicadores.select(
        'cnpj',
        'total_contas',
        'equacao_contabil_perc',
        spark_round('ativo', 2).alias('ativo'),
        spark_round('patrimonio_liquido', 2).alias('pl'),
        spark_round('resultado', 2).alias('resultado'),
        'rentabilidade_pl',
        'endividamento_geral'
    ).orderBy(col('equacao_contabil_perc').asc()) \
     .limit(20) \
     .show(truncate=False)
    
    # ============================================================================
    # RESUMO
    # ============================================================================
    
    print(f"\n" + "=" * 80)
    print("📊 RESUMO FINAL - SCRIPT 2")
    print("=" * 80)
    
    print(f"\n✅ Total de empresas aptas: {total_aptas:,}")
    print(f"✅ Indicadores calculados: {len(indicadores_calculados)}")
    print(f"✅ Tabela: {DATABASE}.ecd_ml_empresas_aptas_indicadores")
    
    print(f"\n📋 Indicadores disponíveis:")
    for i, ind in enumerate(indicadores_calculados, 1):
        print(f"   {i:2d}. {ind}")
    
    print(f"\n🎯 Próximo passo: Execute Script 3 para índices-padrão")

print("\n" + "=" * 80)
print("✅ SCRIPT 2 CONCLUÍDO!")
print("=" * 80)

💼 EMPRESAS APTAS PARA CÁLCULO DE INDICADORES FINANCEIROS

⚙️  Critérios de aptidão:
   - Erro máximo equação: 10.0%
   - Ativo mínimo: R$ 10,000.00

📥 Carregando dados consolidados...
✅ 384,493 empresas carregadas

🔍 Filtrando empresas aptas...
✅ 0 empresas aptas (0.00% do total)

⚠️  NENHUMA EMPRESA APTA ENCONTRADA!

💡 Sugestões:
   1. Aumentar MAX_ERRO_EQUACAO (atual: 10.0%)
   2. Reduzir MIN_ATIVO (atual: R$ 10,000.00)

✅ SCRIPT 2 CONCLUÍDO!


In [39]:
# ================================================================================
# SCRIPT 3: CÁLCULO DE ÍNDICES-PADRÃO POR PORTE
# ================================================================================

print("📊 CÁLCULO DE ÍNDICES-PADRÃO POR PORTE (MÉTODO MATARAZZO)")
print("=" * 80)
print("\n📚 Metodologia: Matarazzo (1998)")

from pyspark.sql.functions import (
    col, when, percentile_approx, count, lit,
    round as spark_round, avg
)
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import pandas as pd

DATABASE = 'neac'
ANO_REFERENCIA = 2024

# Faixas de porte
PORTE_PEQUENO_MAX = 1_000_000
PORTE_MEDIO_MAX = 10_000_000

MIN_EMPRESAS_DECIS = 30

print(f"\n⚙️  Configurações:")
print(f"   - Pequeno: até R$ {PORTE_PEQUENO_MAX:,.0f}")
print(f"   - Médio: até R$ {PORTE_MEDIO_MAX:,.0f}")
print(f"   - Grande: acima de R$ {PORTE_MEDIO_MAX:,.0f}")
print(f"   - Mínimo para decis: {MIN_EMPRESAS_DECIS}")

# ================================================================================
# CARREGAR
# ================================================================================

print(f"\n📥 Carregando...")

df = spark.table(f"{DATABASE}.ecd_ml_empresas_aptas_indicadores")
total_empresas = df.count()

print(f"✅ {total_empresas:,} empresas")

if total_empresas == 0:
    print("\n⚠️  Execute Script 2 primeiro!")
    raise SystemExit

# ================================================================================
# CLASSIFICAR POR PORTE
# ================================================================================

print(f"\n🏢 Classificando por porte...")

df_com_porte = df.withColumn(
    'porte',
    when(col('ativo') <= PORTE_PEQUENO_MAX, 'PEQUENO')
    .when(col('ativo') <= PORTE_MEDIO_MAX, 'MEDIO')
    .otherwise('GRANDE')
)

print(f"\n📊 Distribuição:")
dist_porte = df_com_porte.groupBy('porte').count().orderBy('porte')
dist_porte.show()

dist_dict = {row['porte']: row['count'] for row in dist_porte.collect()}
portes_validos = [porte for porte, qtd in dist_dict.items() if qtd >= MIN_EMPRESAS_DECIS]

if len(portes_validos) == 0:
    print(f"\n⚠️  Nenhum porte tem {MIN_EMPRESAS_DECIS}+ empresas!")
    raise SystemExit

print(f"\n✅ Portes válidos: {portes_validos}")

# ================================================================================
# INDICADORES
# ================================================================================

indicadores_config = {
    'liquidez_corrente': True,
    'liquidez_geral': True,
    'endividamento_geral': False,
    'participacao_capital_terceiros': False,
    'composicao_endividamento': False,
    'margem_liquida': True,
    'rentabilidade_ativo': True,
    'rentabilidade_pl': True,
    'giro_ativo': True
}

indicadores = list(indicadores_config.keys())

print(f"\n📋 Indicadores: {len(indicadores)}")
for ind, maior_melhor in indicadores_config.items():
    sentido = "↑" if maior_melhor else "↓"
    print(f"   {sentido} {ind}")

# ================================================================================
# CALCULAR DECIS
# ================================================================================

print(f"\n📐 CALCULANDO DECIS")
print("=" * 80)

decis_por_porte = {}

for porte in portes_validos:
    print(f"\n🏢 {porte}")
    print("-" * 80)
    
    df_porte = df_com_porte.filter(col('porte') == porte)
    count_porte = df_porte.count()
    
    print(f"   Empresas: {count_porte:,}")
    
    decis_porte = {}
    
    for indicador in indicadores:
        percentis_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        
        result = df_porte.select(
            percentile_approx(indicador, percentis_list, 10000).alias('decis')
        ).collect()
        
        if result and result[0]['decis']:
            decis_porte[indicador] = result[0]['decis']
            print(f"   ✅ {indicador}")
    
    decis_por_porte[porte] = decis_porte

print(f"\n✅ Decis calculados para {len(decis_por_porte)} portes")

# ================================================================================
# EXIBIR TABELAS
# ================================================================================

print(f"\n📋 TABELAS DE ÍNDICES-PADRÃO")
print("=" * 80)

for porte in portes_validos:
    if porte not in decis_por_porte:
        continue
        
    decis_porte = decis_por_porte[porte]
    
    print(f"\n🏢 PORTE: {porte}")
    print("-" * 120)
    print(f"{'Indicador':<35} {'1º':>8} {'2º':>8} {'3º':>8} {'4º':>8} {'5º':>8} {'6º':>8} {'7º':>8} {'8º':>8} {'9º':>8}")
    print("-" * 120)
    
    for indicador in indicadores:
        if indicador in decis_porte:
            decis = decis_porte[indicador]
            valores_str = [f"{v:8.2f}" if v is not None else "     N/A" for v in decis]
            print(f"{indicador:<35} {' '.join(valores_str)}")

# ================================================================================
# CLASSIFICAR EMPRESAS
# ================================================================================

print(f"\n🎯 CLASSIFICANDO EMPRESAS")
print("=" * 80)

def classificar_valor_nos_decis(valor, decis, quanto_maior_melhor):
    if valor is None or decis is None or len(decis) < 9:
        return "NAO_CLASSIFICADO"
    
    posicao = 10
    for i, limite_decil in enumerate(decis):
        if limite_decil is not None and valor <= limite_decil:
            posicao = i + 1
            break
    
    if quanto_maior_melhor:
        if posicao <= 2:
            return "PESSIMO"
        elif posicao <= 3:
            return "DEFICIENTE"
        elif posicao == 4:
            return "FRACO"
        elif posicao == 5:
            return "RAZOAVEL"
        elif posicao in [6, 7]:
            return "SATISFATORIO"
        elif posicao == 8:
            return "BOM"
        else:
            return "OTIMO"
    else:
        if posicao >= 9:
            return "PESSIMO"
        elif posicao == 8:
            return "DEFICIENTE"
        elif posicao == 7:
            return "FRACO"
        elif posicao == 6:
            return "RAZOAVEL"
        elif posicao in [4, 5]:
            return "SATISFATORIO"
        elif posicao == 3:
            return "BOM"
        else:
            return "OTIMO"

dfs_classificados = []

for porte in portes_validos:
    if porte not in decis_por_porte:
        continue
    
    print(f"\n📊 Classificando: {porte}")
    
    df_porte = df_com_porte.filter(col('porte') == porte)
    decis_porte = decis_por_porte[porte]
    
    for indicador in indicadores:
        if indicador not in decis_porte:
            continue
        
        decis = decis_porte[indicador]
        quanto_maior_melhor = indicadores_config[indicador]
        
        @udf(returnType=StringType())
        def classificar_udf(valor):
            if valor is None:
                return "NAO_CLASSIFICADO"
            return classificar_valor_nos_decis(float(valor), decis, quanto_maior_melhor)
        
        nome_coluna_conceito = f'{indicador}_conceito'
        df_porte = df_porte.withColumn(nome_coluna_conceito, classificar_udf(col(indicador)))
    
    dfs_classificados.append(df_porte)

if len(dfs_classificados) > 0:
    df_final_classificado = dfs_classificados[0]
    for df_adicional in dfs_classificados[1:]:
        df_final_classificado = df_final_classificado.union(df_adicional)
    
    print(f"\n✅ Total classificadas: {df_final_classificado.count():,}")
    
    # ============================================================================
    # SALVAR
    # ============================================================================
    
    print(f"\n💾 Salvando...")
    
    df_final_classificado.createOrReplaceTempView("temp_empresas_classificadas")
    
    spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_empresas_indices_padrao")
    
    spark.sql(f"""
        CREATE TABLE {DATABASE}.ecd_ml_empresas_indices_padrao
        STORED AS PARQUET
        AS SELECT * FROM temp_empresas_classificadas
    """)
    
    print(f"✅ Salvo: {DATABASE}.ecd_ml_empresas_indices_padrao")
    
    # ============================================================================
    # EXEMPLOS
    # ============================================================================
    
    print(f"\n📊 EXEMPLOS")
    print("=" * 80)
    
    for porte in portes_validos[:1]:
        print(f"\n🏢 PORTE: {porte}")
        
        df_exemplo = df_final_classificado.filter(col('porte') == porte)
        
        colunas_exemplo = ['cnpj', 'ativo']
        for ind in ['liquidez_corrente', 'rentabilidade_pl', 'endividamento_geral']:
            if f'{ind}_conceito' in df_exemplo.columns:
                colunas_exemplo.extend([ind, f'{ind}_conceito'])
        
        df_exemplo.select(colunas_exemplo).limit(10).show(truncate=False)
    
    # ============================================================================
    # SALVAR DECIS
    # ============================================================================
    
    print(f"\n💾 Salvando tabela de decis...")
    
    decis_data = []
    for porte, decis_porte in decis_por_porte.items():
        for indicador, decis in decis_porte.items():
            decis_data.append({
                'porte': porte,
                'indicador': indicador,
                'decil_1': decis[0] if len(decis) > 0 else None,
                'decil_2': decis[1] if len(decis) > 1 else None,
                'decil_3': decis[2] if len(decis) > 2 else None,
                'decil_4': decis[3] if len(decis) > 3 else None,
                'decil_5': decis[4] if len(decis) > 4 else None,
                'decil_6': decis[5] if len(decis) > 5 else None,
                'decil_7': decis[6] if len(decis) > 6 else None,
                'decil_8': decis[7] if len(decis) > 7 else None,
                'decil_9': decis[8] if len(decis) > 8 else None
            })
    
    df_decis_pandas = pd.DataFrame(decis_data)
    df_decis_spark = spark.createDataFrame(df_decis_pandas)
    
    df_decis_spark.createOrReplaceTempView("temp_decis")
    
    spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_indices_padrao_decis")
    
    spark.sql(f"""
        CREATE TABLE {DATABASE}.ecd_ml_indices_padrao_decis
        STORED AS PARQUET
        AS SELECT * FROM temp_decis
    """)
    
    print(f"✅ Decis salvos: {DATABASE}.ecd_ml_indices_padrao_decis")
    
    # ============================================================================
    # RESUMO
    # ============================================================================
    
    print(f"\n" + "=" * 80)
    print("📊 RESUMO FINAL - SCRIPT 3")
    print("=" * 80)
    
    print(f"\n✅ Empresas classificadas: {df_final_classificado.count():,}")
    print(f"✅ Portes analisados: {len(portes_validos)}")
    print(f"✅ Indicadores: {len(indicadores)}")
    
    print(f"\n📋 Tabelas:")
    print(f"   1. {DATABASE}.ecd_ml_empresas_indices_padrao")
    print(f"   2. {DATABASE}.ecd_ml_indices_padrao_decis")

print("\n" + "=" * 80)
print("✅ SCRIPT 3 CONCLUÍDO!")
print("=" * 80)

📊 CÁLCULO DE ÍNDICES-PADRÃO POR PORTE (MÉTODO MATARAZZO)

📚 Metodologia: Matarazzo (1998)

⚙️  Configurações:
   - Pequeno: até R$ 1,000,000
   - Médio: até R$ 10,000,000
   - Grande: acima de R$ 10,000,000
   - Mínimo para decis: 30

📥 Carregando...
✅ 38 empresas

🏢 Classificando por porte...

📊 Distribuição:
+-------+-----+
|  porte|count|
+-------+-----+
| GRANDE|    2|
|  MEDIO|   18|
|PEQUENO|   18|
+-------+-----+


⚠️  Nenhum porte tem 30+ empresas!


SystemExit: 

### COBERTURA DO MODELO

In [40]:
# ================================================================================
# DIAGNÓSTICO COMPLETO - VERSÃO CORRIGIDA
# ================================================================================

print("🔍 DIAGNÓSTICO: COBERTURA DAS CLASSIFICAÇÕES ML")
print("=" * 80)

from pyspark.sql.functions import col, count, sum as spark_sum, when, abs as spark_abs

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'
ANO_REFERENCIA = 2024

# ================================================================================
# 1. COMPARAR TOTAL DE CONTAS
# ================================================================================

print(f"\n📊 1. COMPARANDO TOTAL DE CONTAS")
print("-" * 80)

# Total de contas no ECD (Balanço)
total_ecd_bp = spark.sql(f"""
    SELECT COUNT(*) as total
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
""").collect()[0]['total']

# Total de predições
total_pred = spark.sql(f"""
    SELECT COUNT(*) as total
    FROM {DATABASE}.ecd_ml_predictions_ALL
""").collect()[0]['total']

# Total ÚNICO
total_ecd_unicas = spark.sql(f"""
    SELECT COUNT(DISTINCT bp.id_ecd, bp.cod_agl) as total
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
""").collect()[0]['total']

print(f"Total de registros no ECD (BP):        {total_ecd_bp:,}")
print(f"Total de contas ÚNICAS (id_ecd+conta): {total_ecd_unicas:,}")
print(f"Total de predições do modelo:          {total_pred:,}")
print(f"DIFERENÇA (contas sem predição):       {total_ecd_unicas - total_pred:,}")
print(f"COBERTURA:                             {total_pred/total_ecd_unicas*100:.2f}%")

# ================================================================================
# 2. ANALISAR CONTAS NÃO CLASSIFICADAS
# ================================================================================

print(f"\n📊 2. ANALISANDO CONTAS NÃO CLASSIFICADAS")
print("-" * 80)

df_nao_classificadas = spark.sql(f"""
    SELECT 
        bp.id_ecd,
        bp.cod_agl as cd_conta,
        bp.descr_cod_agl as descr_conta,
        bp.vl_cta_fin as vl_conta_final,
        bp.ind_dc_cta_fin as ind_dc_final,
        bp.ind_grp_bal,
        pc.cd_natureza,
        pc.tp_conta
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
    INNER JOIN {DATABASE_ECD}.ecd_ri050_plano_contas pc
        ON bp.id_ecd = pc.id_ecd
        AND bp.cod_agl = pc.cd_conta_anl
    LEFT JOIN {DATABASE}.ecd_ml_predictions_ALL pred
        ON bp.id_ecd = pred.id_ecd
        AND bp.cod_agl = pred.cd_conta
    WHERE pred.id_ecd IS NULL
""")

total_nao_class = df_nao_classificadas.count()
print(f"Contas SEM classificação ML: {total_nao_class:,}")

if total_nao_class > 0:
    # Calcular valor total - CORRIGIDO
    df_nao_class_com_valor = df_nao_classificadas.withColumn(
        'vl_ajustado',
        when(col('ind_dc_final') == 'D', col('vl_conta_final'))
        .when(col('ind_dc_final') == 'C', -col('vl_conta_final'))
        .otherwise(col('vl_conta_final'))
    )
    
    # CORREÇÃO: usar Python abs(), não spark abs()
    valor_total_resultado = df_nao_class_com_valor.select(
        spark_sum(spark_abs(col('vl_ajustado'))).alias('total')
    ).collect()[0]
    
    valor_total_nao_class = valor_total_resultado['total'] if valor_total_resultado['total'] else 0
    
    print(f"Valor total (abs) não classificado: R$ {float(valor_total_nao_class):,.2f}")
    
    # Amostra
    print(f"\n📋 Amostra de contas NÃO classificadas:")
    df_nao_classificadas.select(
        'cd_conta', 'descr_conta', 'vl_conta_final', 'ind_grp_bal', 'cd_natureza', 'tp_conta'
    ).limit(20).show(truncate=False)
    
    # Distribuição por tipo
    print(f"\n📊 Distribuição por tipo de conta:")
    df_nao_classificadas.groupBy('tp_conta').count().orderBy(col('count').desc()).show()
    
    # Distribuição por grupo
    print(f"\n📊 Distribuição por grupo do balanço:")
    df_nao_classificadas.groupBy('ind_grp_bal').count().show()

# ================================================================================
# 3. INVESTIGAR POR QUE TÃO POUCAS CONTAS FORAM CLASSIFICADAS
# ================================================================================

print(f"\n📊 3. INVESTIGANDO CAUSA DA BAIXA COBERTURA")
print("-" * 80)

# Ver quais id_ecd foram classificados
empresas_classificadas = spark.sql(f"""
    SELECT COUNT(DISTINCT id_ecd) as total
    FROM {DATABASE}.ecd_ml_predictions_ALL
""").collect()[0]['total']

empresas_total_ecd = spark.sql(f"""
    SELECT COUNT(DISTINCT id_ecd) as total
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial
""").collect()[0]['total']

print(f"Empresas no ECD (BP):           {empresas_total_ecd:,}")
print(f"Empresas com predições:         {empresas_classificadas:,}")
print(f"Cobertura de empresas:          {empresas_classificadas/empresas_total_ecd*100:.2f}%")

# Ver características das contas classificadas vs não classificadas
print(f"\n📊 Características das contas CLASSIFICADAS:")
spark.sql(f"""
    SELECT 
        COUNT(*) as total,
        COUNT(DISTINCT id_ecd) as empresas,
        AVG(LENGTH(cd_conta)) as tamanho_medio_codigo
    FROM {DATABASE}.ecd_ml_predictions_ALL
""").show()

print(f"\n📊 Características das contas NÃO CLASSIFICADAS (amostra):")
df_nao_classificadas.select(
    count('*').alias('total'),
    count(col('id_ecd').isNotNull()).alias('empresas')
).show()

# ================================================================================
# 4. COMPARAR FILTROS
# ================================================================================

print(f"\n📊 4. VERIFICANDO SE HÁ FILTROS APLICADOS")
print("-" * 80)

# Verificar se o modelo foi treinado apenas em SC 2024
print(f"Verificando predições:")
spark.sql(f"""
    SELECT 
        COUNT(DISTINCT id_ecd) as empresas,
        COUNT(DISTINCT cnpj) as cnpjs,
        COUNT(*) as total_predicoes
    FROM {DATABASE}.ecd_ml_predictions_ALL
""").show()

# Verificar dados ECD
print(f"Verificando dados ECD:")
spark.sql(f"""
    SELECT 
        COUNT(DISTINCT id_ecd) as empresas,
        COUNT(DISTINCT cod_agl) as contas_diferentes,
        COUNT(*) as total_registros
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial
""").show()

# ================================================================================
# 5. VERIFICAR SE PREDIÇÕES SÃO APENAS DE UMA AMOSTRA
# ================================================================================

print(f"\n📊 5. VERIFICANDO SE PREDIÇÕES SÃO DE AMOSTRA")
print("-" * 80)

# Ver se há tabela de treino/teste
try:
    total_dataset = spark.sql(f"""
        SELECT COUNT(*) as total
        FROM {DATABASE}.ecd_ml_dataset
    """).collect()[0]['total']
    
    print(f"Total no dataset ML: {total_dataset:,}")
    
    total_test = spark.sql(f"""
        SELECT COUNT(*) as total
        FROM {DATABASE}.ecd_ml_test
    """).collect()[0]['total']
    
    print(f"Total no conjunto de TESTE: {total_test:,}")
    
    print(f"\n⚠️  DESCOBERTA:")
    print(f"   O modelo foi aplicado APENAS no conjunto de TESTE!")
    print(f"   Isso representa {total_test/total_ecd_unicas*100:.2f}% do total")
    
except:
    print("Não encontrou tabela de dataset/test")

print("\n" + "=" * 80)

🔍 DIAGNÓSTICO: COBERTURA DAS CLASSIFICAÇÕES ML

📊 1. COMPARANDO TOTAL DE CONTAS
--------------------------------------------------------------------------------


Total de registros no ECD (BP):        144,602,899
Total de contas ÚNICAS (id_ecd+conta): 80,600,825
Total de predições do modelo:          144,602,905
DIFERENÇA (contas sem predição):       -64,002,080
COBERTURA:                             179.41%

📊 2. ANALISANDO CONTAS NÃO CLASSIFICADAS
--------------------------------------------------------------------------------


Contas SEM classificação ML: 0

📊 3. INVESTIGANDO CAUSA DA BAIXA COBERTURA
--------------------------------------------------------------------------------


Empresas no ECD (BP):           384,493
Empresas com predições:         384,493
Cobertura de empresas:          100.00%

📊 Características das contas CLASSIFICADAS:


+---------+--------+--------------------+
|    total|empresas|tamanho_medio_codigo|
+---------+--------+--------------------+
|144602905|  384493|   9.999866786908603|
+---------+--------+--------------------+


📊 Características das contas NÃO CLASSIFICADAS (amostra):


+-----+--------+
|total|empresas|
+-----+--------+
|    0|       0|
+-----+--------+


📊 4. VERIFICANDO SE HÁ FILTROS APLICADOS
--------------------------------------------------------------------------------
Verificando predições:


+--------+------+---------------+
|empresas| cnpjs|total_predicoes|
+--------+------+---------------+
|  384493|384493|      144602905|
+--------+------+---------------+

Verificando dados ECD:


[Stage 3040:====================================================> (28 + 1) / 29]

+--------+-----------------+---------------+
|empresas|contas_diferentes|total_registros|
+--------+-----------------+---------------+
|  384493|         11382673|      144602899|
+--------+-----------------+---------------+


📊 5. VERIFICANDO SE PREDIÇÕES SÃO DE AMOSTRA
--------------------------------------------------------------------------------
Total no dataset ML: 1,644,053
Total no conjunto de TESTE: 246,359

⚠️  DESCOBERTA:
   O modelo foi aplicado APENAS no conjunto de TESTE!
   Isso representa 0.31% do total



### CLASSIFICAR CONTAS SINTÉTICAS POR HERANÇA

In [41]:
# ================================================================================
# SOLUÇÃO: CLASSIFICAR CONTAS SINTÉTICAS POR HERANÇA
# ================================================================================

print("🔧 CLASSIFICANDO CONTAS SINTÉTICAS POR HERANÇA")
print("=" * 80)

from pyspark.sql.functions import col, when, first, collect_list

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'

# ================================================================================
# 1. IDENTIFICAR CONTAS SINTÉTICAS vs ANALÍTICAS
# ================================================================================

print(f"\n📊 1. IDENTIFICANDO CONTAS SINTÉTICAS")
print("-" * 80)

# Carregar plano de contas
df_plano = spark.sql(f"""
    SELECT 
        id_ecd,
        cd_conta_anl,
        cd_conta_sint,
        nm_conta_anl,
        tp_conta,
        cd_natureza,
        nivel
    FROM {DATABASE_ECD}.ecd_ri050_plano_contas
""")

# Contas sintéticas são aquelas que têm cd_conta_sint preenchido
# E geralmente tp_conta = 'S'
df_sinteticas = df_plano.filter(col('tp_conta') == 'S')
df_analiticas = df_plano.filter(col('tp_conta') == 'A')

total_sint = df_sinteticas.count()
total_anal = df_analiticas.count()

print(f"Contas SINTÉTICAS: {total_sint:,}")
print(f"Contas ANALÍTICAS: {total_anal:,}")

# ================================================================================
# 2. CLASSIFICAR SINTÉTICAS PELA MAIORIA DAS FILHAS
# ================================================================================

print(f"\n🔧 2. CLASSIFICANDO SINTÉTICAS POR HERANÇA")
print("-" * 80)

# Carregar predições existentes
df_pred = spark.table(f"{DATABASE}.ecd_ml_predictions_ALL")

# Join: plano de contas + predições
df_plano_com_pred = df_plano.alias('pc').join(
    df_pred.alias('pred'),
    (col('pc.id_ecd') == col('pred.id_ecd')) & 
    (col('pc.cd_conta_anl') == col('pred.cd_conta')),
    how='left'
)

# Para cada conta sintética, pegar classificação das filhas
classificacao_sinteticas = spark.sql(f"""
    WITH filhas_classificadas AS (
        SELECT 
            pc_sint.id_ecd,
            pc_sint.cd_conta_anl as conta_sintetica,
            pred.classificacao_nivel2,
            COUNT(*) as qtd
        FROM {DATABASE_ECD}.ecd_ri050_plano_contas pc_anal
        INNER JOIN {DATABASE_ECD}.ecd_ri050_plano_contas pc_sint
            ON pc_anal.id_ecd = pc_sint.id_ecd
            AND pc_anal.cd_conta_sint = pc_sint.cd_conta_anl
        INNER JOIN {DATABASE}.ecd_ml_predictions_ALL pred
            ON pc_anal.id_ecd = pred.id_ecd
            AND pc_anal.cd_conta_anl = pred.cd_conta
        WHERE pc_sint.tp_conta = 'S'
        GROUP BY pc_sint.id_ecd, pc_sint.cd_conta_anl, pred.classificacao_nivel2
    ),
    classificacao_majoritaria AS (
        SELECT 
            id_ecd,
            conta_sintetica,
            FIRST(classificacao_nivel2) as classificacao_herdada,
            SUM(qtd) as total_filhas
        FROM filhas_classificadas
        GROUP BY id_ecd, conta_sintetica
        ORDER BY SUM(qtd) DESC
    )
    SELECT * FROM classificacao_majoritaria
""")

total_sint_classificadas = classificacao_sinteticas.count()
print(f"✅ Sintéticas classificadas por herança: {total_sint_classificadas:,}")

# Salvar
classificacao_sinteticas.createOrReplaceTempView("temp_sinteticas_classificadas")

spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_sinteticas_por_heranca")

spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_sinteticas_por_heranca
    STORED AS PARQUET
    AS SELECT * FROM temp_sinteticas_classificadas
""")

print(f"✅ Salvo em: {DATABASE}.ecd_ml_sinteticas_por_heranca")

print("\n" + "=" * 80)

🔧 CLASSIFICANDO CONTAS SINTÉTICAS POR HERANÇA

📊 1. IDENTIFICANDO CONTAS SINTÉTICAS
--------------------------------------------------------------------------------


Contas SINTÉTICAS: 51,198,730
Contas ANALÍTICAS: 457,254,458

🔧 2. CLASSIFICANDO SINTÉTICAS POR HERANÇA
--------------------------------------------------------------------------------


                                                                                0]

✅ Sintéticas classificadas por herança: 8,401,167


[Stage 3126:=================================================>    (12 + 1) / 13]0]

✅ Salvo em: neac.ecd_ml_sinteticas_por_heranca



### CLASSIFICAR CONTAS POR FALLBACK POR NATUREZA

In [42]:
# ================================================================================
# SOLUÇÃO: CLASSIFICAÇÃO FALLBACK POR cd_natureza
# ================================================================================

print("🔧 CLASSIFICAÇÃO FALLBACK PARA CONTAS NÃO CLASSIFICADAS")
print("=" * 80)

from pyspark.sql.functions import col, when, substring

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'

# ================================================================================
# MAPEAR cd_natureza → Classificação
# ================================================================================

print(f"\n📊 Aplicando classificação fallback por cd_natureza...")

# Carregar contas não classificadas
df_nao_class = spark.sql(f"""
    SELECT 
        bp.id_ecd,
        bp.cod_agl as cd_conta,
        bp.descr_cod_agl as descr_conta,
        bp.vl_cta_fin,
        bp.ind_dc_cta_fin,
        bp.ind_grp_bal,
        pc.cd_natureza,
        pc.tp_conta
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
    INNER JOIN {DATABASE_ECD}.ecd_ri050_plano_contas pc
        ON bp.id_ecd = pc.id_ecd
        AND bp.cod_agl = pc.cd_conta_anl
    LEFT JOIN {DATABASE}.ecd_ml_predictions_ALL pred
        ON bp.id_ecd = pred.id_ecd
        AND bp.cod_agl = pred.cd_conta
    WHERE pred.id_ecd IS NULL
""")

# Aplicar regras de fallback baseadas em cd_natureza
df_fallback = df_nao_class.withColumn(
    'classificacao_fallback',
    # ATIVO
    when(col('cd_natureza').startswith('1.01'), 'ATIVO_CIRCULANTE')
    .when(col('cd_natureza').startswith('1.02'), 'ATIVO_NAO_CIRCULANTE')
    .when(col('cd_natureza').startswith('1.'), 'ATIVO')
    # PASSIVO
    .when(col('cd_natureza').startswith('2.01'), 'PASSIVO_CIRCULANTE')
    .when(col('cd_natureza').startswith('2.02'), 'PASSIVO_NAO_CIRCULANTE')
    # PATRIMÔNIO LÍQUIDO
    .when(col('cd_natureza').startswith('2.03'), 'PATRIMONIO_LIQUIDO')
    .when(col('cd_natureza').startswith('2.3'), 'PATRIMONIO_LIQUIDO')
    # Passivo genérico
    .when(col('cd_natureza').startswith('2.'), 'PASSIVO')
    # Baseado em ind_grp_bal se cd_natureza não for claro
    .when((col('ind_grp_bal') == 'A'), 'ATIVO')
    .when((col('ind_grp_bal') == 'P'), 'PASSIVO')
    .otherwise('NAO_CLASSIFICADO')
)

total_fallback = df_fallback.filter(col('classificacao_fallback') != 'NAO_CLASSIFICADO').count()
print(f"✅ Contas classificadas por fallback: {total_fallback:,}")

# Salvar
df_fallback.createOrReplaceTempView("temp_fallback")

spark.sql(f"DROP TABLE IF EXISTS {DATABASE}.ecd_ml_fallback_classificacoes")

spark.sql(f"""
    CREATE TABLE {DATABASE}.ecd_ml_fallback_classificacoes
    STORED AS PARQUET
    AS SELECT * FROM temp_fallback
""")

print(f"✅ Salvo em: {DATABASE}.ecd_ml_fallback_classificacoes")

print("\n" + "=" * 80)

🔧 CLASSIFICAÇÃO FALLBACK PARA CONTAS NÃO CLASSIFICADAS

📊 Aplicando classificação fallback por cd_natureza...


✅ Contas classificadas por fallback: 0


[Stage 3145:===================================================>(199 + 1) / 200]

✅ Salvo em: neac.ecd_ml_fallback_classificacoes



### RECALCULAR EQUAÇÕES COM TODAS AS CLASSIFICAÇÕES

In [43]:
# ================================================================================
# SOLUÇÃO: RECALCULAR EQUAÇÃO CONTÁBIL COM CLASSIFICAÇÕES COMPLETAS
# ================================================================================

print("🔧 RECALCULANDO EQUAÇÃO CONTÁBIL COM CLASSIFICAÇÕES EXPANDIDAS")
print("=" * 80)

from pyspark.sql.functions import col, when, coalesce, sum as spark_sum, abs as spark_abs

DATABASE = 'neac'
DATABASE_ECD = 'usr_sat_ecd'

# ================================================================================
# 1. CONSOLIDAR TODAS AS CLASSIFICAÇÕES
# ================================================================================

print(f"\n📊 1. CONSOLIDANDO CLASSIFICAÇÕES")
print("-" * 80)

# União de todas as fontes de classificação
df_class_completa = spark.sql(f"""
    SELECT 
        bp.id_ecd,
        bp.cod_agl as cd_conta,
        bp.vl_cta_fin,
        bp.ind_dc_cta_fin,
        COALESCE(
            pred.classificacao_nivel2,      -- 1ª prioridade: ML
            sint.classificacao_herdada,     -- 2ª prioridade: Herança
            fb.classificacao_fallback       -- 3ª prioridade: Fallback
        ) as classificacao_final
    FROM {DATABASE_ECD}.ecd_rj100_balanco_patrimonial bp
    LEFT JOIN {DATABASE}.ecd_ml_predictions_ALL pred
        ON bp.id_ecd = pred.id_ecd AND bp.cod_agl = pred.cd_conta
    LEFT JOIN {DATABASE}.ecd_ml_sinteticas_por_heranca sint
        ON bp.id_ecd = sint.id_ecd AND bp.cod_agl = sint.conta_sintetica
    LEFT JOIN {DATABASE}.ecd_ml_fallback_classificacoes fb
        ON bp.id_ecd = fb.id_ecd AND bp.cod_agl = fb.cd_conta
""")

# Contar cobertura
cobertura = df_class_completa.filter(col('classificacao_final').isNotNull()).count()
total = df_class_completa.count()

print(f"Cobertura ANTES: {246359/total*100:.2f}%")
print(f"Cobertura DEPOIS: {cobertura/total*100:.2f}%")
print(f"Melhoria: +{(cobertura-246359)/total*100:.2f} pontos percentuais")

# ================================================================================
# 2. RECALCULAR BALANÇOCOM CLASSIFICAÇÃO COMPLETA
# ================================================================================

# Aplicar mesmo processo do Script 1, mas com classificação_final
# [CÓDIGO DO SCRIPT 1 AQUI, usando classificacao_final]

print("\n✅ Execute o Script 1 novamente com as classificações expandidas!")
print("=" * 80)

🔧 RECALCULANDO EQUAÇÃO CONTÁBIL COM CLASSIFICAÇÕES EXPANDIDAS

📊 1. CONSOLIDANDO CLASSIFICAÇÕES
--------------------------------------------------------------------------------


[Stage 3166:====================================================> (65 + 2) / 67]]

Cobertura ANTES: 0.05%
Cobertura DEPOIS: 100.00%
Melhoria: +99.95 pontos percentuais

✅ Execute o Script 1 novamente com as classificações expandidas!
